In [1]:
import cohere

In [2]:
client = cohere.Client()

In [5]:
response = client.chat(
    message="is it too late to join the course?"
)

In [7]:
response.text

"I don't have enough information to answer your question. It depends on the specific course and its enrollment policies. Some courses may have strict deadlines for registration, while others may allow late additions. It's best to check with the course organizer or institution directly to find out if you can still join. They can provide you with the most accurate and up-to-date information regarding enrollment deadlines and any possible exceptions."